In [7]:
import pandas as pd
import geopandas as gpd
import requests, zipfile, io

In [43]:
# use the csv created from 01_geocode.ipynb
df_geo = pd.read_csv('../data/output/df_geo.csv')

In [45]:
# convert it to a geodataframe for spatial join with council districts
df_gpd = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo['lon'], df_geo['lat']), crs="EPSG:4326")

In [46]:
# get the council district boundaries release 23b from 
# https://www.nyc.gov/site/planning/data-maps/open-data/bytes-archive.page
url = 'https://s-media.nyc.gov/agencies/dcp/assets/files/zip/data-tools/bytes/nycc_23b.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('../data/input/')

cd = gpd.read_file('../data/input/nycc_23b/nycc.shp').to_crs('EPSG:4326')

In [47]:
# make sure crs matches
cd.crs = df_gpd.crs

In [48]:
# join the data from csv with the council district boundaries 
geo_cd = df_gpd.sjoin(cd, how = 'left', predicate = 'intersects')

In [ ]:
# assign cd values to column 'New Council District'
geo_cd['New Council District'] = geo_cd['CounDist']

In [56]:
# write to csv the new data with relevant columns
(pd.DataFrame(geo_cd)
.drop(columns=['geoclient', 'geometry', 'index_right', 'Shape_Leng', 'Shape_Area', 'CounDist'])
.to_csv('../data/output/geo_cd.csv', index=False))